In [1]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from vae_earlystopping import EarlyStopping
from model.m2_mse import MSEcVAE
from loss.l2_mse import l2_mse
import joblib
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import json

In [3]:
# 결과 저장용
results = {
    "R2_": [],
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_and_evaluate(c_path, x_data, seed):
    """
    특정 조건 데이터(c_path)를 받아 BCE+MSE 결합 모델의 R2 Score를 반환하는 함수
    """
    c_data = np.load(c_path)
    
    # 데이터 분할
    x_train, x_temp, c_train, c_temp = train_test_split(x_data, c_data, random_state=seed, test_size=0.4)
    x_val, x_test, c_val, c_test = train_test_split(x_temp, c_temp, random_state=seed, test_size=0.5)
    
    # 스케일링
    x_scaler, c_scaler = MinMaxScaler(), MinMaxScaler()
    x_train = x_scaler.fit_transform(x_train)
    c_train = c_scaler.fit_transform(c_train)
    x_val, x_test = x_scaler.transform(x_val), x_scaler.transform(x_test)
    c_val, c_test = c_scaler.transform(c_val), c_scaler.transform(c_test)
    
    # Loader 생성
    t_loader = DataLoader(TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(c_train)), batch_size=64, shuffle=True)
    v_loader = DataLoader(TensorDataset(torch.FloatTensor(x_val), torch.FloatTensor(c_val)), batch_size=64)
    test_loader = DataLoader(TensorDataset(torch.FloatTensor(x_test), torch.FloatTensor(c_test)), batch_size=64)
    
    x_dim, c_dim = x_train.shape[1], c_train.shape[1]

    # --- MSE 모델 학습 ---
    model_mse = MSEcVAE(x_dim, c_dim, z_dim=8).to(device)
    opt_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
    es_mse = EarlyStopping(patience=40, min_delta=1e-9)
    
    for epoch in range(1, 801):
        model_mse.train()
        for x, c in t_loader:
            x, c = x.to(device), c.to(device)
            opt_mse.zero_grad()
            xh, mu, logvar = model_mse(x, c)
            loss = l2_mse(xh, x, mu, logvar)['loss']
            loss.backward(); opt_mse.step()
            
        model_mse.eval()
        val_loss = 0
        with torch.no_grad():
            for vx, vc in v_loader:
                vx, vc = vx.to(device), vc.to(device)
                xh, vm, vv = model_mse(vx, vc)
                val_loss += l2_mse(xh, vx, vm, vv)['loss'].item()
        if es_mse(val_loss/len(v_loader), model_mse): break
        
    es_mse.load_best_model(model_mse)

    # --- 3. 최종 평가 (BCE_MSE 가중합) ---
    model_mse.eval()
    mse_preds, x_true = [], []
    
    with torch.no_grad():
        for xt, ct in test_loader:
            xt, ct = xt.to(device), ct.to(device)
            m_pred, _, _ = model_mse(xt, ct)
            mse_preds.append(m_pred.cpu().numpy())
            x_true.append(xt.cpu().numpy())
            
    x_hat = x_scaler.inverse_transform(np.vstack(mse_preds))
    x_true = x_scaler.inverse_transform(np.vstack(x_true))
    
    # Soft weighting 적용
    return r2_score(x_true.flatten(), x_hat.flatten())

# --- 메인 실험 루프 ---
x_data = np.load('./data/metal.npy')
seeds = np.random.randint(1, 100, size=20)

for i, seed in enumerate(seeds):
    print(f"\n[Iteration {i+1}/20] Seed: {seed}")
    
    
    # Case 2: Without Reaction
    r2 = train_and_evaluate('./data/pre_re_change_temp_logconst.npy', x_data, seed)
    results["R2_"].append(float(r2))
    
    print(f" R2: {r2:.4f}")

# 결과 저장
with open("./compare_results.json", "w") as f:
    json.dump(results, f, indent=2)


[Iteration 1/20] Seed: 91
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out

In [4]:
save_path = "./results_regression_logreaction_constant.json"
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Saved:", save_path)

Saved: ./results_regression_logreaction_constant.json
